In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
# os.environ["WORLD_SIZE"] = "1"

# Download and convert data

In [4]:
from tqdm import tqdm

from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysis
from document_segmentation.settings import RENATE_ANALYSIS_DIR

N = None

RENATE_ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)

sheet = RenateAnalysis()


existing_docs = {
    path.stem for path in RENATE_ANALYSIS_DIR.glob("Globdoc_*.json") if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),
    desc="Writing documents",
    unit="doc",
):
    document_file = RENATE_ANALYSIS_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents: 100%|██████████| 78/78 [03:15<00:00,  2.51s/doc]


In [5]:
from tqdm import tqdm

from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysisInv
from document_segmentation.settings import RENATE_ANALYSIS_DIR, RENATE_ANALYSIS_SHEETS

N = None


sheet = RenateAnalysisInv(RENATE_ANALYSIS_SHEETS[0])  # TODO: use both sheets

existing_docs = {
    path.stem
    for path in RENATE_ANALYSIS_DIR.glob("NL-HaNA_1.04.02_*.json")
    if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),  # FIXME: len(sheet) is wrong
    desc="Writing documents",
    unit="doc",
):
    document_file = RENATE_ANALYSIS_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents:   0%|          | 0/690 [00:00<?, ?doc/s]

Writing documents:   4%|▍         | 26/690 [00:11<04:46,  2.32doc/s]


# Load Data

In [6]:
%autoreload now

In [7]:
TRAINING_DATA = 0.8

In [8]:
from document_segmentation.model.dataset import PageDataset
from document_segmentation.settings import MIN_REGION_TEXT_LENGTH

dataset = PageDataset.from_dir(RENATE_ANALYSIS_DIR).remove_short_regions(
    MIN_REGION_TEXT_LENGTH
)
len(dataset)

Reading JSON files: 100%|██████████| 104/104 [00:01<00:00, 86.56file/s] 


2184

In [9]:
dataset._class_counts()

Counter({<Label.IN: 2>: 1907,
         <Label.BEGIN: 1>: 104,
         <Label.END: 3>: 100,
         <Label.OUT: 4>: 73})

In [10]:
dataset.class_weights()

[20.8, 1.1446540880503144, 21.623762376237625, 29.513513513513512]

In [12]:
split = int(len(dataset) * TRAINING_DATA)

training_data = dataset[:split]
training_data._class_counts()

Counter({<Label.IN: 2>: 1533,
         <Label.BEGIN: 1>: 77,
         <Label.END: 3>: 75,
         <Label.OUT: 4>: 62})

In [13]:
test_data = dataset[split:]
test_data._class_counts()

Counter({<Label.IN: 2>: 374,
         <Label.BEGIN: 1>: 27,
         <Label.END: 3>: 25,
         <Label.OUT: 4>: 11})

# Train Model

In [14]:
import torch

BATCH_SIZE = 32
EPOCHS = 3
WEIGHTS = torch.Tensor(dataset.class_weights())  # For an imbalanced dataset

In [15]:
%autoreload now

In [16]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [17]:
tagger._device

'mps'

In [18]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_features=4, bias=True)
  (_soft

In [19]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS.to(tagger._device))

 99%|█████████▉| 54/54.59375 [01:30<00:00,  1.46s/batch]/Users/carstenschnober/LAHTeR/workspace/document-segmentation/.venv/lib/python3.11/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
101%|██████████| 55/54.59375 [01:31<00:00,  1.66s/batch]


Current allocated memory (MPS): 1170 MB
Driver allocated memory (MPS): 2887 MB
[Loss:	1.163]


101%|██████████| 55/54.59375 [00:03<00:00, 13.83batch/s]


Current allocated memory (MPS): 1162 MB
Driver allocated memory (MPS): 2967 MB
[Loss:	1.165]


101%|██████████| 55/54.59375 [00:03<00:00, 13.88batch/s]

Current allocated memory (MPS): 1162 MB
Driver allocated memory (MPS): 2851 MB
[Loss:	1.095]


# Evaluate Model

In [20]:
import csv
import sys

from torcheval.metrics import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall,
)
from tqdm import tqdm

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout,
    fieldnames=("Predicted", "Actual", "Page ID", "Text", "Scores"),
    delimiter="\t",
)

writer.writeheader()

accuracy = MulticlassAccuracy(num_classes=len(Label))
precision = MulticlassPrecision(average=None, num_classes=len(Label))
recall = MulticlassRecall(average=None, num_classes=len(Label))
f1_score = MulticlassF1Score(average=None, num_classes=len(Label))

for batch in tqdm(
    test_data.batches(BATCH_SIZE), total=len(test_data) / BATCH_SIZE, unit="batch"
):
    predicted = tagger(batch)
    labels = batch.labels()

    _labels = torch.Tensor([label.value - 1 for label in labels]).to(int)
    accuracy.update(predicted, _labels)
    precision.update(predicted, _labels)
    recall.update(predicted, _labels)
    f1_score.update(predicted, _labels)

    for page, pred, label in zip(batch.pages, predicted, labels):
        pred_label = Label(pred.argmax().item() + 1)
        if pred_label != Label.IN or label != Label.IN:
            writer.writerow(
                {
                    "Predicted": pred_label.name,
                    "Actual": label.name,
                    "Page ID": page.doc_id,
                    "Text": page.text(delimiter="; ")[:50],
                    "Scores": str(pred.tolist()),
                }
            )

Predicted	Actual	Page ID	Text	Scores


  7%|▋         | 1/13.65625 [00:00<00:10,  1.21batch/s]

END	IN	NL-HaNA_1.04.02_3524_0986.jpg	van 10000 kyserdaalders ter leen zullende hij alle	[0.12512490153312683, 0.30856895446777344, 0.5628307461738586, 0.00347542786039412]
END	IN	NL-HaNA_1.04.02_3524_0987.jpg	Hier meede deesen bekortende maatigen wij ons de; 	[0.06166611984372139, 0.08441950380802155, 0.851412832736969, 0.0025015235878527164]
END	END	NL-HaNA_1.04.02_3524_0988.jpg	Notitie van Zodaanige Coopmansz:; etc: als Sariepr	[0.003870869055390358, 0.005092552397400141, 0.9894911050796509, 0.0015453965170308948]
OUT	OUT	NL-HaNA_1.04.02_1547_0481.jpg		[0.0005603920435532928, 0.0002981974685098976, 0.003644691314548254, 0.9954967498779297]
OUT	OUT	NL-HaNA_1.04.02_1547_0482.jpg		[0.0008527957834303379, 0.00024216008023358881, 0.0014517775271087885, 0.9974531531333923]
BEGIN	BEGIN	NL-HaNA_1.04.02_1547_0483.jpg	Saterdagh den 4=en xber; Present den Commandeur; P	[0.9136616587638855, 0.04032803326845169, 0.03998707979917526, 0.006023257505148649]
BEGIN	IN	NL-HaNA_1.04.02_1547_0484.jpg	na 

 15%|█▍        | 2/13.65625 [00:02<00:18,  1.58s/batch]

IN	END	NL-HaNA_1.04.02_1547_0467.jpg	Waer seven daegen zoo aen mijn lighaem te verschoo	[0.11907272785902023, 0.7566057443618774, 0.12283354252576828, 0.001488027162849903]
IN	BEGIN	NL-HaNA_1.04.02_1268_1095.jpg	Is nae aen roepingh van godes Heijsige naeme; vers	[0.11873476952314377, 0.7482216954231262, 0.13157106935977936, 0.0014724008506163955]
IN	END	NL-HaNA_1.04.02_1268_1109.jpg	g'admitteert en tot godes kerke aengenomen werden 	[0.11895566433668137, 0.7560752034187317, 0.12349596619606018, 0.0014732044655829668]
IN	BEGIN	NL-HaNA_1.04.02_3446_0595.jpg	Aan De Edele Hoog Agtb„e Heeren; Bewindhebberen va	[0.11902810633182526, 0.7544757723808289, 0.12500500679016113, 0.001491079805418849]
IN	END	NL-HaNA_1.04.02_3446_0603.jpg	Batavia; in 't Casteel den; 25:e october; waar mee	[0.12001077830791473, 0.661048412322998, 0.2169722318649292, 0.001968554686754942]
IN	BEGIN	NL-HaNA_1.04.02_2542_0114.jpg	Van Mallabaar onder dato; 9. stux te weeten; 6. ma	[0.13269509375095367, 0.5253822207450867,

...

END	IN	NL-HaNA_1.04.02_3248_0899.jpg	Waar mede den relatant dit zijne gegevene relaas c	[0.018082819879055023, 0.019184289500117302, 0.9540315866470337, 0.008701292797923088]
OUT	IN	NL-HaNA_1.04.02_3248_0900.jpg		[0.0009222699445672333, 0.00037951141712255776, 0.0023496318608522415, 0.9963486194610596]
BEGIN	IN	NL-HaNA_1.04.02_3248_0901.jpg	Op heden den 8:e Februarij 1768: Compa„; „reerde v	[0.7902325987815857, 0.09338448196649551, 0.10774803906679153, 0.008634885773062706]
END	END	NL-HaNA_1.04.02_3248_0902.jpg	waar mede den relatant zijn gegevene relaas quam t	[0.38019809126853943, 0.18494176864624023, 0.4292825162410736, 0.005577538628131151]
END	BEGIN	NL-HaNA_1.04.02_1547_0107.jpg	Extracten uijt de Daagelijkse Aanteeckeningen, Con	[0.10026136785745621, 0.0484062023460865, 0.8476727604866028, 0.0036596853751689196]
END	END	NL-HaNA_1.04.02_1547_0108.jpg	op desen holvn en dass: matthijs in ve totenende a	[0.003603915683925152, 0.0031431664247065783, 0.9913488030433655, 0.00190408644266

 95%|█████████▌| 13/13.65625 [00:32<00:01,  2.82s/batch]

IN	END	NL-HaNA_1.04.02_1083_0048.jpg	Des avondts voor het aronsteten is den predicanten	[0.11937545984983444, 0.7591578960418701, 0.12000031024217606, 0.0014663924230262637]
IN	BEGIN	NL-HaNA_1.04.02_8696_0051.jpg	Van Siam onder dato 20: april 1737; Dag-register, 	[0.12174913287162781, 0.7546426057815552, 0.122219018638134, 0.0013892798451706767]


103%|██████████| 14/13.65625 [00:33<00:00,  2.41s/batch]

IN	END	NL-HaNA_1.04.02_8696_0077.jpg	Van Siam onder dato: 20: april 1737; Translaet Sia	[0.13300307095050812, 0.5275892615318298, 0.3366990089416504, 0.0027086276095360518]


In [21]:
writer = csv.DictWriter(
    sys.stdout,
    fieldnames=["Metric", "Average"] + [label.name for label in Label],
    delimiter="\t",
)
writer.writeheader()

for metric in (precision, recall, f1_score):
    scores = {
        label.name: f"{score:.4f}"
        for label, score in zip(Label, metric.compute().tolist())
    }
    writer.writerow(
        {"Metric": metric.__class__.__name__, "Average": str(metric.average)} | scores
    )

print(f"Accuracy ({accuracy.average} average):\t{accuracy.compute().item():.4f}")

Metric	Average	BEGIN	IN	END	OUT
MulticlassPrecision	None	0.1739	0.9045	0.2051	0.2750
MulticlassRecall	None	0.1481	0.8102	0.3200	1.0000
MulticlassF1Score	None	0.1600	0.8547	0.2500	0.4314
Accuracy (micro average):	0.7460
